Все импорты должны выполняться, особенно если вы запускаете это внутри докера

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [18]:
import pandas as pd
import pandas.io.sql as psql

Вспомогательные функции для запроса данных из базы

In [2]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql("SELECT uid FROM users WHERE data_loaded")

In [3]:
# Всего у нас вот столько таблиц

run_sql("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")

,table_name
0,groups
1,likes
2,posts
3,users_friends
4,users_groups
5,users


In [4]:
run_sql("Select * FROM users where data_loaded limit 5")

,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,photo_100,photo_200_orig,can_write_private_message,status,data_loaded,mobile_phone,home_phone,group_list_loaded,comments_loaded,is_leader
uid,,,,,,,,,,,,,,,,,,,
72914186,Lada ...,21099i,2,,id72914186,11.3,0,1,https://pp.userapi.com/c4979/u72914186/e_1f6aa...,https://pp.userapi.com/c4979/u72914186/d_1e185...,https://pp.userapi.com/c4979/u72914186/a_2b50a...,0,,True,89092319524,89092319524,True,True,0
88282133,Рашид ...,Гусейнов,2,,id88282133,,2,1,https://pp.userapi.com/c9229/u88282133/e_c7d52...,https://pp.userapi.com/c9229/u88282133/d_f940e...,https://pp.userapi.com/c9229/u88282133/a_9d7f4...,1,,True,,,True,True,0
78145940,Roma ...,Lkl,2,,id78145940,15.3.1989,2,1,https://pp.userapi.com/c659/u78145940/e_2ffe27...,https://pp.userapi.com/c659/u78145940/d_acedc3...,https://pp.userapi.com/c659/u78145940/a_6e05a7...,1,,True,,,True,True,0
33666831,Аня ...,Круткина,1,,anu.tkina,5.9,1,1,https://pp.userapi.com/c636629/v636629831/13a4...,https://pp.userapi.com/c636629/v636629831/13a4...,https://pp.userapi.com/c636629/v636629831/13a3...,1,,True,,,True,True,0
33580978,Настя ...,Головина,1,,id33580978,29.5.1993,0,0,https://pp.userapi.com/c850024/v850024756/67c0...,https://pp.userapi.com/c850024/v850024756/67c0...,https://pp.userapi.com/c850024/v850024756/67c0...,0,,True,,,True,True,0


In [35]:
users = run_sql("""Select uid, first_name, last_name, sex, is_leader 
                FROM users 
                where data_loaded and group_list_loaded and comments_loaded""")

users.to_csv('../data/raw_data/users.csv', index=True)

In [6]:
run_sql("""Select * 
           FROM groups 
           limit 5""")

,gid,name,screen_name,is_closed,type,photo,photo_medium,photo_big
0,48457106,ТМ ВиТиеВаТо / Украшения из проволоки,tm_vitievato,0,page,https://pp.userapi.com/c633231/v633231909/17c3...,https://pp.userapi.com/c633231/v633231909/17c3...,https://pp.userapi.com/c633231/v633231909/17c3...
1,52159629,Реальные Пацаны (Новые Серии) | Универ (Новый),tnt_realnue,0,page,https://pp.userapi.com/mv-k7Kdy6ulFCEW6sO2bmuN...,https://pp.userapi.com/cZtc5wEUIXozLniDSp8Uf9o...,https://pp.userapi.com/TffOP3rm5582WEUn7tnedwx...
2,52914122,Ростов-город ● 161 ● Ростов-Дон,rostov_gorod_161_rostov_don,0,page,https://pp.userapi.com/c841329/v841329164/3091...,https://pp.userapi.com/c841329/v841329164/3091...,https://pp.userapi.com/c841329/v841329164/3091...
3,53376961,HOT WHEELS Exclusive Сar Shop,hwexclusive,0,page,https://pp.userapi.com/5OLTm6mWr9CJVOGfg53tQfN...,https://pp.userapi.com/rEhCO9T1y4vZfrC4JTEgwW_...,https://pp.userapi.com/Jwqeyx2QPcfquCtLLy7sHBB...
4,53995491,Booking Machine Shop,bookingmachineshop,0,page,https://sun1-86.userapi.com/c837139/v837139778...,https://sun1-14.userapi.com/c837139/v837139778...,https://sun1-21.userapi.com/c837139/v837139778...


In [7]:
df = run_sql("""Select gid, name, screen_name 
                FROM groups 
                where is_closed = 0""")

df.to_csv('../data/raw_data/groups.csv', index=False)

In [8]:
run_sql("""Select * 
           FROM likes 
           limit 5""")

,id,owner_id,user_id
0,3694,63350736,179907918
1,3958,101192512,335342017
2,205,8520297,4511856
3,3840,101192512,10453460
4,2142,97043772,439713007


In [9]:
df = run_sql("""Select owner_id, user_id 
                FROM likes""")

df = df.groupby('owner_id', as_index=False).agg('count')
df.to_csv('../data/raw_data/likes.csv', index=False)

In [10]:
run_sql("""Select * 
           FROM users_friends 
           limit 5""")

,uid1,uid2
0,320805228,107620371
1,107620371,320805228
2,320805228,107701389
3,107701389,320805228
4,320805228,108109373


In [11]:
df = run_sql("""Select uid1, count(uid2) 
                FROM users_friends
                group by uid1""")


In [12]:
df.to_csv('../data/raw_data/friends.csv', index=False)

In [13]:
run_sql("""Select * 
           FROM users_groups 
           limit 5""")

,gid
uid,
4080406,131631279
4080406,149349881
4080406,35193970
4080406,27121021
4080406,10280580


In [14]:
df = run_sql("""Select * 
                FROM users_groups 
             """)

In [15]:
df.to_csv('../data/raw_data/users_groups.csv', index=False)

In [16]:
run_sql("""Select * 
           FROM posts
           where text <> ''
           limit 5
        """)

,id,from_id,owner_id,date,post_type,text,post_source,comments,is_favorite,likes,reposts,copy_history,attachments,likes_loaded
0,9607,166834,166834,1461438105,post,ВидеоОбзор#2 - Худший РЭПЕР Словетский (Газгол...,{'type': 'vk'},"{'count': 0, 'can_post': 0, 'groups_can_post':...",False,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{},"[{'type': 'video', 'video': {'id': 456239136, ...",True
1,9350,166834,166834,1460461201,post,ИДЕАЛЬНЫЕ МОШЕННИКИ,{'type': 'vk'},"{'count': 0, 'can_post': 0, 'groups_can_post':...",False,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{},"[{'type': 'video', 'video': {'id': 456239111, ...",True
2,9484,166834,166834,1461156201,post,1. Японцы употребляют очень мало жира и меньше...,{'type': 'vk'},"{'count': 0, 'can_post': 0, 'groups_can_post':...",False,"{'count': 2, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{},"[{'type': 'photo', 'photo': {'id': 410074650, ...",True
3,9423,166834,166834,1460886113,post,[Ретроспектива Сокола] Терминатор,{'type': 'vk'},"{'count': 0, 'can_post': 0, 'groups_can_post':...",False,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{},"[{'type': 'video', 'video': {'id': 456239120, ...",True
4,9412,166834,166834,1460813204,post,СУТЬ ПРЯМОЙ ЛИНИИ ПУТИНА 2016,{'type': 'vk'},"{'count': 0, 'can_post': 0, 'groups_can_post':...",False,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{},"[{'type': 'video', 'video': {'id': 456239118, ...",True


In [19]:
from tqdm import tqdm

total = 8982138
def dict2int(x):
    return x['count']

posts = []

for i in tqdm(range((total//1000000) + 1)):

    df = run_sql("""Select from_id, owner_id, text, comments, likes, reposts
                    FROM posts
                    where text <> '' and likes_loaded
                    limit 1000000 offset {}
                 """.format(i*1000000))

    df['comments'] = df['comments'].apply(dict2int).astype(np.int32)
    df['likes'] = df['likes'].apply(dict2int).astype(np.int32)
    df['reposts'] = df['reposts'].apply(dict2int).astype(np.int32)
    posts.append(df)

df = pd.concat(posts).drop_duplicates()
df.to_csv('../data/raw_data/posts.csv', index=False)

In [20]:
df = pd.concat(posts)
df.drop_duplicates(inplace=True)

In [21]:
df.shape

(3862035, 6)

In [31]:
df.iloc[df.comments.argmax()].text

'ЧЁт уТро ваЩе нЕ дОбРое...'

In [ ]:
df.shape

In [ ]:
df.to_csv('../data/raw_data/posts_1kk.csv', index=False)

In [ ]:
1

In [ ]:
# from psycopg2 import sql

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#     sql.SQL("ALTER TABLE {} DROP COLUMN is_leader")
#         .format(sql.Identifier('users')))
# conn.commit()



In [ ]:
# # Добавляем новую колонку напрямую в базу


# from psycopg2 import sql

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#     sql.SQL("ALTER TABLE {} ADD COLUMN is_leader INT")
#         .format(sql.Identifier('users')))
# conn.commit()


In [32]:
# Допустим теперь для каждого юзера у нас появились предсказания, запишем их в сабмишн файл 

ids = get_uid()
leaders = np.random.choice(list(ids.index), size=100)


submission = pd.DataFrame({'is_leader':np.zeros(ids.shape[0])}, index=ids.index)
submission.loc[leaders] = 1

submission.to_csv('../data/submission/random_submission.csv')

In [ ]:
# # Обновляем колонку в базе 
# # Так делать не нужно, код для Андрея
# from tqdm import tqdm

# conn = get_conn()
# cur = conn.cursor()

# cur.execute(
#         sql.SQL("""UPDATE {} SET is_leader = 0""")
#     .format(sql.Identifier('users'))
# )

# leaders = submission[submission['is_leader'] == 1]
# for uid, is_leader in tqdm(zip(leaders.index, leaders['is_leader'])):
#     cur.execute(
#         sql.SQL("""UPDATE {}
#                 SET is_leader = %s
#                 WHERE
#                    uid = %s;
#                 """)
#             .format(sql.Identifier('users')),
#         [int(is_leader), uid])
# conn.commit()
